In [1]:
# LIBRARIES
import requests
import pandas as pd
import re

In [2]:
## API URL
url = 'https://dadosabertos.camara.leg.br/api/v2/deputados?itens=1000&ordem=ASC&ordenarPor=nome'

## Request
r = requests.get(url)

## create a dataframe
df = pd.DataFrame(r.json()['dados'])

def get_last_page(url):
    r2 = requests.get(url)    
    links = pd.DataFrame(r2.json()['links'])
    count_links = len(links)
    return count_links
    
## lists to store the data and the ids
list = []
id_list = []
name_list = []
partido_list = []
uf_list = []
urlFoto_list = []
email_list = []



for year in range(2020, 2022+1):
    page = 1
    count = 0

    ## request the data for each id
    for id in df['id']:    
        url = 'https://dadosabertos.camara.leg.br/api/v2/deputados/'+ str(id) + '/despesas?ano=' + str(year) + '&pagina=' + str(page) + '&itens=1000&ordem=ASC&ordenarPor=ano'  
        last_page = get_last_page(url)     
        count += 1
        
        ## pagination
        for page in range(1, int(last_page)+1):
            url = 'https://dadosabertos.camara.leg.br/api/v2/deputados/'+ str(id) + '/despesas?ano=' + str(year) + '&pagina=' + str(page) + '&itens=1000&ordem=ASC&ordenarPor=ano'
            r2 = requests.get(url)
            df_despezas = pd.json_normalize(r2.json()['dados'])

            ## counting how many times the ID appears
            count_data = len(r2.json()['dados'])
            
            
            ## create a list with the ID and the number of times it appears
            for i in range(count_data):
                id_list.append(id)
                name_list.append(df['nome'][count-1])
                partido_list.append(df['siglaPartido'][count-1])
                uf_list.append(df['siglaUf'][count-1])
                urlFoto_list.append(df['urlFoto'][count-1])
                email_list.append(df['email'][count-1])

         
        
            ## create a list with the data for each ID
            list.append(df_despezas)

## create a dataframe with the data from the list
df_despezastotal = pd.concat(list)

## add the ID column to the dataframe
df_despezastotal['id'] = id_list
df_despezastotal['nome'] = name_list
df_despezastotal['siglaPartido'] = partido_list
df_despezastotal['siglaUf'] = uf_list
df_despezastotal['urlFoto'] = urlFoto_list
df_despezastotal['email'] = email_list

## export the dataframe to a csv file
df_despezastotal.to_csv('despesas_deputados_teste.csv', index=False)